In [8]:
import os
import pypdf

def extract_text_from_pdf(pdf_path):
    """ 從 PDF 提取文本內容 """
    text = ""
    with open(pdf_path, "rb") as file:
        reader = pypdf.PdfReader(file)
        for page in reader.pages:
            text += page.extract_text() + "\n"
    return text


In [9]:
# Load model directly
# from transformers import AutoTokenizer, AutoModel
from sentence_transformers import SentenceTransformer

model = SentenceTransformer("sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2")

def text_to_vector(text):
    """ 將文本轉換為向量 """
    return model.encode(text)


In [10]:
import chromadb

# 初始化 ChromaDB 並創建 Collection
chroma_client = chromadb.PersistentClient(path="./chroma_db")
collection = chroma_client.get_or_create_collection(name="pdf_documents")


In [11]:
def store_pdf_in_chroma(pdf_path, doc_id):
    """ 讀取 PDF、轉換為向量並存儲到 ChromaDB """
    text = extract_text_from_pdf(pdf_path)
    vector = text_to_vector(text)

    collection.add(
        ids=[doc_id],  # 唯一識別碼
        embeddings=[vector.tolist()],  # 向量化結果
        metadatas=[{"filename": os.path.basename(pdf_path), "text": text}]  # 儲存部分文字內容
    )

# 批量處理多個 PDF 文件
pdf_folder = "..\\product infomation"
pdf_files = [f for f in os.listdir(pdf_folder) if f.endswith(".pdf")]

for idx, pdf_file in enumerate(pdf_files):
    store_pdf_in_chroma(os.path.join(pdf_folder, pdf_file), doc_id=f"doc_{idx}")


Add of existing embedding ID: doc_0
Insert of existing embedding ID: doc_0
Add of existing embedding ID: doc_1
Insert of existing embedding ID: doc_1
Add of existing embedding ID: doc_2
Insert of existing embedding ID: doc_2
Add of existing embedding ID: doc_3
Insert of existing embedding ID: doc_3
Add of existing embedding ID: doc_4
Insert of existing embedding ID: doc_4


In [12]:
def search_similar_pdfs(query_text, top_k=3):
    """ 搜尋與輸入文本最相似的 PDF 文件 """
    query_vector = text_to_vector(query_text)
    
    results = collection.query(
        query_embeddings=[query_vector.tolist()],
        n_results=top_k
    )
    
    return results["documents"], results["metadatas"], results["distances"]

# 測試檢索
query = "耳機"
docs, metas, dists = search_similar_pdfs(query)

for doc, meta, dist in zip(docs[0], metas[0], dists[0]):
    print(f"文件: {meta['filename']} (相似度: {1 - dist:.4f})\n內容預覽: {meta['text']}\n")


文件: 羅技 Logitech H340 USB耳機麥克風.pdf (相似度: -9.4967)
內容預覽: 商品名稱羅技 Logitech H340 USB耳機麥克風 店家名稱筑蒂資訊有限公司
商品售價945 店家電話04-22216131
店家地址台灣省台中市中山路49巷11-2號
商品描述
商品資料表
H340 USB Computer Headset
UBS隨插即用
H340 USB 耳機麥克風具有簡易隨插即用 USB-A 
連線功能，是完美的日常工作耳機麥克風。隔噪麥克風可減少背景噪音，為 VO

文件: SADES DIABLO 暗黑鬥狼RGB REALTEK 電競耳麥 7.1 (USB) SA-916.pdf (相似度: -13.7832)
內容預覽: 商品名稱SADES DIABLO 暗黑鬥狼RGB REALTEK 電競耳麥 7.1 (USB) SA-916店家名稱筑蒂資訊有限公司
商品售價2,390 店家電話04-22216131
店家地址台灣省台中市中山路49巷11-2號
商品描述
商品資料表
<<黑暗中閃耀的不只是我的利爪 還有你即將流下的眼淚>> 
• 全球首發Realtek瑞昱電競耳機音效系統 
• 電競音效功能：擴闊音場/強化通話/

文件: W202 人體工學椅.pdf (相似度: -18.9989)
內容預覽: 商品名稱W202 人體工學椅 店家名稱Xpanse旗艦店
商品售價1,460 店家電話未提供
店家地址未提供
商品描述
商品資料表
耐重粗鐵管腳+強化塑料框架 結構穩實乘坐安全可靠
椅背採優選透氣網布，韌性高易拉伸不易斯裂
高密度定型棉坐墊，柔軟舒適久坐
當空間較小時或需搬動時可以瞬間收納，不佔空間
㇐體式折合桌椅輕便重量只有8KG,收納移動簡單便利 
已投保1000萬-產品責任險~讓您安心使用多



In [14]:
query = "辦公桌"
docs, metas, dists = search_similar_pdfs(query)

for doc, meta, dist in zip(docs[0], metas[0], dists[0]):
    print(f"文件: {meta['filename']} (相似度: {1 - dist:.4f})\n內容預覽: {meta['text']}\n")

文件: [折疊收納]懶人折疊桌.pdf (相似度: -19.9151)
內容預覽: 商品名稱[折疊收納]懶人折疊桌 店家名稱吉來小舖
商品售價95 店家電話未提供
店家地址未提供
商品描述
商品資料表
產品名稱：懶人折疊桌
產品顏色：木紋
展開尺寸：50x30x23cm
折疊尺寸：50x30x5cm
產品重量：約1290克
產品材質：密度板+鋼管+塑料
產品用途：折疊桌
組裝需求：無
使用方法：展開即可使用
產品產地：中國
產品貨源：原廠公司貨
保存方法：本產品請避光陰涼處存放。

文件: W202 人體工學椅.pdf (相似度: -21.8566)
內容預覽: 商品名稱W202 人體工學椅 店家名稱Xpanse旗艦店
商品售價1,460 店家電話未提供
店家地址未提供
商品描述
商品資料表
耐重粗鐵管腳+強化塑料框架 結構穩實乘坐安全可靠
椅背採優選透氣網布，韌性高易拉伸不易斯裂
高密度定型棉坐墊，柔軟舒適久坐
當空間較小時或需搬動時可以瞬間收納，不佔空間
㇐體式折合桌椅輕便重量只有8KG,收納移動簡單便利 
已投保1000萬-產品責任險~讓您安心使用多

文件: 羅技 Logitech H340 USB耳機麥克風.pdf (相似度: -23.5981)
內容預覽: 商品名稱羅技 Logitech H340 USB耳機麥克風 店家名稱筑蒂資訊有限公司
商品售價945 店家電話04-22216131
店家地址台灣省台中市中山路49巷11-2號
商品描述
商品資料表
H340 USB Computer Headset
UBS隨插即用
H340 USB 耳機麥克風具有簡易隨插即用 USB-A 
連線功能，是完美的日常工作耳機麥克風。隔噪麥克風可減少背景噪音，為 VO



In [15]:
query = "工作桌"
docs, metas, dists = search_similar_pdfs(query)

for doc, meta, dist in zip(docs[0], metas[0], dists[0]):
    print(f"文件: {meta['filename']} (相似度: {1 - dist:.4f})\n內容預覽: {meta['text']}\n")

文件: [折疊收納]懶人折疊桌.pdf (相似度: -18.0205)
內容預覽: 商品名稱[折疊收納]懶人折疊桌 店家名稱吉來小舖
商品售價95 店家電話未提供
店家地址未提供
商品描述
商品資料表
產品名稱：懶人折疊桌
產品顏色：木紋
展開尺寸：50x30x23cm
折疊尺寸：50x30x5cm
產品重量：約1290克
產品材質：密度板+鋼管+塑料
產品用途：折疊桌
組裝需求：無
使用方法：展開即可使用
產品產地：中國
產品貨源：原廠公司貨
保存方法：本產品請避光陰涼處存放。

文件: W202 人體工學椅.pdf (相似度: -19.8583)
內容預覽: 商品名稱W202 人體工學椅 店家名稱Xpanse旗艦店
商品售價1,460 店家電話未提供
店家地址未提供
商品描述
商品資料表
耐重粗鐵管腳+強化塑料框架 結構穩實乘坐安全可靠
椅背採優選透氣網布，韌性高易拉伸不易斯裂
高密度定型棉坐墊，柔軟舒適久坐
當空間較小時或需搬動時可以瞬間收納，不佔空間
㇐體式折合桌椅輕便重量只有8KG,收納移動簡單便利 
已投保1000萬-產品責任險~讓您安心使用多

文件: 羅技 Logitech H340 USB耳機麥克風.pdf (相似度: -21.5987)
內容預覽: 商品名稱羅技 Logitech H340 USB耳機麥克風 店家名稱筑蒂資訊有限公司
商品售價945 店家電話04-22216131
店家地址台灣省台中市中山路49巷11-2號
商品描述
商品資料表
H340 USB Computer Headset
UBS隨插即用
H340 USB 耳機麥克風具有簡易隨插即用 USB-A 
連線功能，是完美的日常工作耳機麥克風。隔噪麥克風可減少背景噪音，為 VO



In [ ]:
query = "電競的耳麥"
docs, metas, dists = search_similar_pdfs(query, 5)

for doc, meta, dist in zip(docs[0], metas[0], dists[0]):
    print(f"文件: {meta['filename']} (相似度: {1 - dist:.4f})\n內容預覽: {meta['text']}\n")

文件: 羅技 Logitech H340 USB耳機麥克風.pdf (相似度: -6.9175)
內容預覽: 商品名稱羅技 Logitech H340 USB耳機麥克風 店家名稱筑蒂資訊有限公司
商品售價945 店家電話04-22216131
店家地址台灣省台中市中山路49巷11-2號
商品描述
商品資料表
H340 USB Computer Headset
UBS隨插即用
H340 USB 耳機麥克風具有簡易隨插即用 USB-A 
連線功能，是完美的日常工作耳機麥克風。隔噪麥克風可減少背景噪音，為 VO

文件: SADES DIABLO 暗黑鬥狼RGB REALTEK 電競耳麥 7.1 (USB) SA-916.pdf (相似度: -7.6960)
內容預覽: 商品名稱SADES DIABLO 暗黑鬥狼RGB REALTEK 電競耳麥 7.1 (USB) SA-916店家名稱筑蒂資訊有限公司
商品售價2,390 店家電話04-22216131
店家地址台灣省台中市中山路49巷11-2號
商品描述
商品資料表
<<黑暗中閃耀的不只是我的利爪 還有你即將流下的眼淚>> 
• 全球首發Realtek瑞昱電競耳機音效系統 
• 電競音效功能：擴闊音場/強化通話/

文件: [折疊收納]懶人折疊桌.pdf (相似度: -11.5355)
內容預覽: 商品名稱[折疊收納]懶人折疊桌 店家名稱吉來小舖
商品售價95 店家電話未提供
店家地址未提供
商品描述
商品資料表
產品名稱：懶人折疊桌
產品顏色：木紋
展開尺寸：50x30x23cm
折疊尺寸：50x30x5cm
產品重量：約1290克
產品材質：密度板+鋼管+塑料
產品用途：折疊桌
組裝需求：無
使用方法：展開即可使用
產品產地：中國
產品貨源：原廠公司貨
保存方法：本產品請避光陰涼處存放。

文件: W202 人體工學椅.pdf (相似度: -11.7959)
內容預覽: 商品名稱W202 人體工學椅 店家名稱Xpanse旗艦店
商品售價1,460 店家電話未提供
店家地址未提供
商品描述
商品資料表
耐重粗鐵管腳+強化塑料框架 結構穩實乘坐安全可靠
椅背採優選透氣網布，韌性高易拉伸不易斯裂
高密度定型棉坐墊，柔軟舒適久坐
當空間較小時或需搬動時可以瞬間收納，不佔空間
㇐體式折合桌椅輕便重量只有8KG,收納移動簡單便利

In [22]:
with open("a.txt", "w", encoding="utf-8") as f:
    for doc, meta, dist in zip(docs[0], metas[0], dists[0]):
        f.write(f"文件: {meta['filename']} (相似度: {1 - dist:.4f})\n內容預覽:\n{meta['text']}\n\n")

In [23]:
query = "工作桌"
docs, metas, dists = search_similar_pdfs(query, 5)

with open("a.txt", "w", encoding="utf-8") as f:
    for doc, meta, dist in zip(docs[0], metas[0], dists[0]):
        f.write(f"文件: {meta['filename']} (相似度: {1 - dist:.4f})\n內容預覽:\n{meta['text']}\n\n")